En este archivo se desarrolla una interfaz en la que se carga una base de datos, se implementan los tres modelos de machine learning y se obtienen las métricas de evaluación para determinar cuál es el mejor modelo para dicho químico o escenario.

In [2]:
from tkinter import *
from tkinter import filedialog
from doepy import build, read_write
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron


Aquí se definen las diferentes funciones a utilizar para el preprocesamiento de los datos, el estudio de correlación y la implementación de los diferentes modelos de machine learning.

In [4]:

global data 
data = pd.DataFrame({'A' : [1, 2, 3, 4]})

def browseFiles():
    filename = filedialog.askopenfilename(
        initialdir = "/",
        title = "Select a File",
        filetypes = (("Text files","*.csv*"),)
    )
    # Change label contents
    label_file_explorer.configure(text="Selected file: "+filename)
    global data
    data = pd.read_csv(filename)
    read_write.write_csv(data,'DataBase')
    data = CleanData(data)
    print('done')
    data = generateDummies(data)  
    print('done')  
    read_write.write_csv(data,'correlation')
     

def CleanData(table):
    table.pop('Y complement')
    table.pop('R complement')
    table.pop('O complement')
    sameColumns = table.loc[:,[(table[col] == table[col][0]).all() for col in table.columns]]
    for i in range(len(sameColumns.columns)):
        table.pop(sameColumns.columns[i])
    print(len(table))
    table = table.drop(range(190,len(table)))
    return table

def generateDummies(table) :
    dummieColumns = table.loc[:,(table.dtypes == 'object').values]
    for i in range(len(dummieColumns.columns)):
        print(dummieColumns.columns[i])
        dummieEncode = pd.get_dummies(table[dummieColumns.columns[i]], prefix='x')
        table.pop(dummieColumns.columns[i])
        table = table.join(dummieEncode)
    return table

def pearsonCorr() :
    
    

    #fig = plt.figure(figsize=(19, 15))
    
    corr_matrix = data.corr(method='pearson')
    corr_matrix.style.background_gradient(cmap='coolwarm').set_precision(2)


    


def spearmanCorr() : 
    corr_matrix = data.corr(method='spearman')
    corr_matrix.style.background_gradient(cmap='coolwarm').set_precision(2)

def kendallCorr() : 
    corr_matrix = data.corr(method='kendall')
    corr_matrix.style.background_gradient(cmap='coolwarm').set_precision(2)

def KNNmodel() :
    global data
    X = data.drop(['red distance (km)', 'yellow distance', 'orange distance'], axis=1)
    y1 = data['yellow distance']
    y1 = y1.values
    y2 = data['orange distance']
    y2 = y2.values
    y3 = data['red distance (km)']
    y3 = y3.values
    rmse1 = 0
    rmse2 = 0
    rmse3 = 0
    mape1 = []
    mape2 = []
    mape3 = []
    y1test = []
    preds1 = []
    y2test = []
    preds2 = []
    y3test = []
    preds3 = []
    for x in range(10):
        X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.1,)
        X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.1,)
        X3_train, X3_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.1,)

        y1test.extend(y1_test)
        y2test.extend(y2_test)
        y3test.extend(y3_test)

        knn_model1 = KNeighborsRegressor(n_neighbors=10)
        knn_model1.fit(X1_train, y1_train)
        knn_model2 = KNeighborsRegressor(n_neighbors=10)
        knn_model2.fit(X2_train, y2_train)
        knn_model3 = KNeighborsRegressor(n_neighbors=10)
        knn_model3.fit(X3_train, y3_train)

        test_preds1 = knn_model1.predict(X1_test)
        mse1 = mean_squared_error(y1_test, test_preds1)
        rmse1 = (rmse1 +sqrt(mse1))/2
        test_preds2 = knn_model2.predict(X2_test)
        mse2 = mean_squared_error(y2_test, test_preds2)
        rmse2 = (rmse2 + sqrt(mse2))/2
        test_preds3 = knn_model3.predict(X3_test)
        mse3 = mean_squared_error(y3_test, test_preds3)
        rmse3 = (rmse3 + sqrt(mse3))/2

        errors1 = abs(test_preds1 - y1_test)
        mape1.extend(100 * (errors1 / y1_test))
        
        errors2 = abs(test_preds2 - y2_test)
        mape2.extend(100 * (errors2 / y2_test))
        
        errors3 = abs(test_preds3 - y3_test)
        mape3.extend(100 * (errors3 / y3_test))
        
        preds1.extend(test_preds1)
        preds2.extend(test_preds2)
        preds3.extend(test_preds3)

    print('mape')
    print(mape3)
    accuracy1 = 100 - np.mean(mape1)
    accuracy2 = 100 - np.mean(mape2)
    accuracy3 = 100 - np.mean(mape3)
    print('accuracy')
    print(accuracy3)
    print('amarillo')
    print(preds1)
    print(y1test)
    print('naranja')
    print(preds2)
    print(y2test)
    print('rojo')
    print(preds3)
    print(y3test)
    """ global fig1
    global fig2
    global fig3
    print(y1test[0:20])
    plot1 = fig1.add_subplot(111)
    plot1.plot( y1test[0:40], 'b-', label = 'actual')
    plot1.set_title('Yellow distance')
    plot1.plot( preds1[0:40], 'ro', label = 'prediction')
    plt.legend()

    plot2 = fig2.add_subplot(111)
    plot2.plot( y2test[0:40], 'b-', label = 'actual')
    plot2.set_title('Orange distance')
    plot2.plot( preds2[0:40], 'ro', label = 'prediction')
    plt.legend()

    plot3 = fig3.add_subplot(111)
    plot3.plot( y3test[0:40], 'b-', label = 'actual')
    plot3.set_title('Red distance')
    plot3.plot( preds3[0:40], 'ro', label = 'prediction')
    plt.legend()


    canvas1 = FigureCanvasTkAgg(fig1, master = graphFrame) 
    canvas1.draw()
    canvas1.get_tk_widget().place(x=20, y=50)"""
    label_result1.configure(text="RMSE: " + str(rmse1) + ".\n" "Accuracy:" + str(accuracy1))

    """ canvas2 = FigureCanvasTkAgg(fig2, master = graphFrame)  
    canvas2.draw()
    canvas2.get_tk_widget().place(x=340, y=50) """
    label_result2.configure(text="RMSE: " + str(rmse2) + ".\n" "Accuracy:" + str(accuracy2))

    """ canvas3 = FigureCanvasTkAgg(fig3, master = graphFrame)  
    canvas3.draw()
    canvas3.get_tk_widget().place(x=660, y=50) """
    label_result3.configure(text="RMSE: " + str(rmse3) + ".\n" "Accuracy:" + str(accuracy3)) 

def RFModel() :
    global data
    labels1 = np.array(data['yellow distance'])
    labels2 = np.array(data['orange distance'])
    labels3 = np.array(data['red distance (km)'])
    features= data.drop(['red distance (km)', 'yellow distance', 'orange distance'], axis=1)
    rmse1 = 0
    rmse2 = 0
    rmse3 = 0
    mape1 = []
    mape2 = []
    mape3 = []
    y1test = []
    preds1 = []
    y2test = []
    preds2 = []
    y3test = []
    preds3 = []
    features = np.array(features)

    for x in range(10):

        train1_features, test1_features, train1_labels, test1_labels = train_test_split(features, labels1, test_size = 0.10,)
        train2_features, test2_features, train2_labels, test2_labels = train_test_split(features, labels2, test_size = 0.10,)
        train3_features, test3_features, train3_labels, test3_labels = train_test_split(features, labels3, test_size = 0.10,)

        y1test.append(test1_labels)
        y2test.append(test2_labels)
        y3test.append(test3_labels)

        rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, min_samples_leaf=10)
        rf.fit(train1_features, train1_labels)
        rf.fit(train2_features, train2_labels)
        rf.fit(train3_features, train3_labels)

        # Use the forest's predict method on the test data
        predictions1 = rf.predict(test1_features)
        predictions2 = rf.predict(test2_features)
        predictions3 = rf.predict(test3_features)
    
        # Calculate and display accuracy
        mse1 = mean_squared_error(predictions1, test1_labels)
        rmse1 = rmse1 + sqrt(mse1)
        mse2 = mean_squared_error(predictions2, test2_labels)
        rmse2 = rmse2 + sqrt(mse2)
        mse3 = mean_squared_error(predictions3, test3_labels)
        rmse3 = rmse3 + sqrt(mse3)
        
        errors1 = abs(predictions1 - test1_labels)
        mape1.extend(100 * (errors1 / test1_labels))
        
        errors2 = abs(predictions2 - test2_labels)
        mape2.extend(100 * (errors2 / test2_labels))
        
        errors3 = abs(predictions3 - test2_labels)
        mape3.extend(100 * (errors3 / test2_labels))

        preds1.append(predictions1)
        preds2.append(predictions2)
        preds3.append(predictions3)

    global fig1
    global fig2
    global fig3

    accuracy1 = 100 - np.mean(mape1)
    accuracy2 = 100 - np.mean(mape2)
    accuracy3 = 100 - np.mean(mape3)
    print(accuracy3)

    """ plot1 = fig1.add_subplot(111)
    plot1.plot( y1test[0:50], 'b-', label = 'actual')
    plot1.set_title('Yellow distance')
    plot1.plot( preds1[0:50], 'r-', label = 'prediction')

    plot2 = fig2.add_subplot(111)
    plot2.plot( y2test[0:50], 'b-', label = 'actual')
    plot2.set_title('Orange distance')
    plot2.plot( preds2[0:50], 'r-', label = 'prediction')

    plot3 = fig3.add_subplot(111)
    plot3.plot( y3test[0:50], 'b-', label = 'actual')
    plot3.set_title('Red distance')
    plot3.plot( preds3[0:50], 'r-', label = 'prediction')


    canvas1 = FigureCanvasTkAgg(fig1, master = graphFrame) 
    canvas1.draw()
    canvas1.get_tk_widget().place(x=20, y=50) """
    label_result1.configure(text="RMSE: " + str(rmse1) + str(rmse1) + ".\n" "Accuracy:" + str(accuracy1))

    """ canvas2 = FigureCanvasTkAgg(fig2, master = graphFrame)  
    canvas2.draw()
    canvas2.get_tk_widget().place(x=340, y=50) """
    label_result2.configure(text="RMSE: " + str(rmse2) + str(rmse1) + ".\n" "Accuracy:" + str(accuracy2))

    """ canvas3 = FigureCanvasTkAgg(fig3, master = graphFrame)  
    canvas3.draw()
    canvas3.get_tk_widget().place(x=660, y=50) """
    label_result3.configure(text="RMSE: " + str(rmse3) + + str(rmse1) + ".\n" "Accuracy:" + str(accuracy3))

def MLPModel():
    lab = preprocessing.LabelEncoder()
    data['yellow distance'] = lab.fit_transform(data['yellow distance'])
    data['orange distance'] = lab.fit_transform(data['orange distance'])
    data['red distance (km)'] = lab.fit_transform(data['red distance (km)'])

    # Spliting data into Feature and
    y1 = np.array(data['yellow distance'])
    y2 = np.array(data['orange distance'])
    y3 = np.array(data['red distance (km)'])
    X = data.drop('yellow distance', axis = 1)
    X = data.drop('orange distance', axis = 1)
    X = data.drop('red distance (km)', axis = 1)
    rmse1 = 0
    rmse2 = 0
    rmse3 = 0
    mape1 = []
    mape2 = []
    mape3 = []
    y1test = []
    preds1 = []
    y2test = []
    preds2 = []
    y3test = []
    preds3 = []

    for x in range(10):

        # Split dataset into training set and test set
        train1_features, test1_features, train1_labels, test1_labels = train_test_split(X, y1, test_size=0.1,)
        train2_features, test2_features, train2_labels, test2_labels = train_test_split(X, y2, test_size=0.1,)
        train3_features, test3_features, train3_labels, test3_labels = train_test_split(X, y3, test_size=0.1,)
        
        y1test.append(test1_labels)
        y2test.append(test2_labels)
        y3test.append(test3_labels)
        
        # Create model object
        clf1 = MLPClassifier(random_state = 457, hidden_layer_sizes= (10,100))
        clf1.fit(train1_features,train1_labels)
        clf2 = MLPClassifier(random_state = 457)
        clf2.fit(train2_features,train2_labels)
        clf3 = MLPClassifier(random_state = 457)
        clf3.fit(train3_features,train3_labels)

        # Make prediction on test dataset
        ypred1 = clf1.predict(test1_features)
        ypred2 = clf2.predict(test2_features)
        ypred3 = clf3.predict(test3_features)

        # rmse

        mse1 = mean_squared_error(ypred1, test1_labels)
        rmse1 = rmse1 + sqrt(mse1)
        mse2 = mean_squared_error(ypred2, test2_labels)
        rmse2 = rmse2 + sqrt(mse2)
        mse3 = mean_squared_error(ypred3, test3_labels)
        rmse3 = rmse3 + sqrt(mse3)

        errors1 = abs(ypred1 - test1_labels)
        mape1.extend(100 * (errors1 / test1_labels))
        
        errors2 = abs(ypred2 - test2_labels)
        mape2.extend(100 * (errors2 / test2_labels))
        
        errors3 = abs(ypred3 - test3_labels)
        mape3.extend(100 * (errors3 / test3_labels))

        preds1.append(ypred1)
        preds2.append(ypred2)
        preds3.append(ypred3)

    global fig1
    global fig2
    global fig3

    accuracy1 = 100 - np.mean(mape1)
    accuracy2 = 100 - np.mean(mape2)
    accuracy3 = 100 - np.mean(mape3)
    print(accuracy3)

    """ plot1 = fig1.add_subplot(111)
    plot1.plot( y1test[0:50], 'b-', label = 'actual')
    plot1.set_title('Yellow distance')
    plot1.plot( preds1[0:50], 'r-', label = 'prediction')

    plot2 = fig2.add_subplot(111)
    plot2.plot( y2test[0:50], 'b-', label = 'actual')
    plot2.set_title('Orange distance')
    plot2.plot( preds2[0:50], 'r-', label = 'prediction')

    plot3 = fig3.add_subplot(111)
    plot3.plot( y3test[0:50], 'b-', label = 'actual')
    plot3.set_title('Red distance')
    plot3.plot( preds3[0:50], 'r-', label = 'prediction')


    canvas1 = FigureCanvasTkAgg(fig1, master = graphFrame) 
    canvas1.draw()
    canvas1.get_tk_widget().place(x=20, y=50) """
    label_result1.configure(text="RMSE: " + str(rmse1) + str(rmse1) + ".\n" "Accuracy:" + str(accuracy1))

    """ canvas2 = FigureCanvasTkAgg(fig2, master = graphFrame)  
    canvas2.draw()
    canvas2.get_tk_widget().place(x=340, y=50) """
    label_result2.configure(text="RMSE: " + str(rmse2) + str(rmse1) + ".\n" "Accuracy:" + str(accuracy2))

    """ canvas3 = FigureCanvasTkAgg(fig3, master = graphFrame)  
    canvas3.draw()
    canvas3.get_tk_widget().place(x=660, y=50) """
    label_result3.configure(text="RMSE: " + str(rmse3) + str(rmse1) + ".\n" "Accuracy:" + str(accuracy3))



## Creaciónd e la interfaz con tkinter

window = Tk()
window.title('Machine Learning Simulator')
window.geometry("1000x730")
window.config(background = "grey")

titleFrame = Frame(window, height = 40, width = 990,)
titleFrame.place(x=500, y=25, anchor="center")

title = Label(
    titleFrame,
    text = "Machin Learning Simulator",
    fg = "black",
    font = ("Arial", 15),

)
title.place(x=500, y=20, anchor="center")

fileFrame = Frame(window, height = 75, width = 990, )
fileFrame.place(x=500, y=85, anchor="center")

label1 = Label(
    fileFrame,
    text = "Please, select the database",
    fg = "black",
    font = ("Arial", 11),
)
label1.place(x=450, y=20, anchor="center")

button_explore = Button(
    fileFrame,
    text = "Browse csv File",
    bg = "white",
    command = browseFiles
)
button_explore.place(x=600, y=20, anchor="center")

label_file_explorer = Label(
    fileFrame,
    text = "Seleted file:",
) 
label_file_explorer.place(x=50, y= 40)

corrFrame = Frame(window, height = 90, width = 493,)
corrFrame.place(x=251, y=171, anchor="center")

mlFrame = Frame(window, height = 90, width = 493,)
mlFrame.place(x=748, y=171, anchor="center")

# correlation buttons

corrLabel = Label(
    corrFrame,
    text = "Correlation options:",
)
corrLabel.place(x=30, y= 10)

button_Pearson = Button(
    corrFrame,
    text = "Pearson Correlation",
    command = pearsonCorr
)
button_Pearson.place(x= 55, y= 40)

button_Spearman = Button(
    corrFrame,
    text = "Spearman Correlation",
    command = spearmanCorr
)
button_Spearman.place(x= 185, y= 40)

button_Kendall = Button(
    corrFrame,
    text = "Kendall Correlation",
    command = kendallCorr
)
button_Kendall.place(x= 330, y= 40)

# ML buttons

mlLabel = Label(
    mlFrame,
    text = "Machine Learning options:",
)
mlLabel.place(x=30 , y= 10)

button_KNN = Button(
    mlFrame,
    text = "KNN Model",
    command = KNNmodel
) 
button_KNN.place(x= 50, y=40)

button_SVM = Button(
    mlFrame,
    text = "SVM Model",
    command = KNNmodel
) 
button_SVM.place(x= 160, y=40)

button_RF = Button(
    mlFrame,
    text = "RF Model",
    command = RFModel
) 
button_RF.place(x= 265, y=40)

button_MLP = Button(
    mlFrame,
    text = "MLP Model",
    command = MLPModel
) 
button_MLP.place(x= 360, y=40)

## graph

graphFrame = Frame(window, height = 500, width = 990)
graphFrame.place(x= 5, y=220)

fig1 = Figure(figsize = (3, 3),dpi = 100)
fig2 = Figure(figsize = (3, 3),dpi = 100)
fig3 = Figure(figsize = (3, 3),dpi = 100)

#canvas1 = FigureCanvasTkAgg(fig1, master = graphFrame)  

label_result1 = Label(
    graphFrame,
    text = "",
    font = ("Arial", 12),
) 
label_result1.place(x= 60, y=370)

label_result2 = Label(
    graphFrame,
    text = "",
    font = ("Arial", 12),
) 
label_result2.place(x= 390, y=370)

label_result3 = Label(
    graphFrame,
    text = "",
    font = ("Arial", 12),
) 
label_result3.place(x= 700, y=370)


window.mainloop()

300
done
Building
Building surrounding
Ground roughtness
Hole or Valve
shape (circular or rectangular)
model run
done
60.132036330582835


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1776.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\maria\AppData\Local\Temp\ipykernel_4836\500388423.py", line 276, in RFModel
    label_result3.configure(text="RMSE: " + str(rmse3) + + str(rmse1) + ".\n" "Accuracy:" + str(accuracy3))
TypeError: bad operand type for unary +: 'str'


58.41855732865972


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1776.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\maria\AppData\Local\Temp\ipykernel_4836\500388423.py", line 276, in RFModel
    label_result3.configure(text="RMSE: " + str(rmse3) + + str(rmse1) + ".\n" "Accuracy:" + str(accuracy3))
TypeError: bad operand type for unary +: 'str'
